In [ ]:
import os
import sys
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['THEANO_FLAGS'] = 'device=gpu,compiledir_format="ipynb_compiledir_%(platform)s-%(processor)s-%(python_version)s-%(python_bitwidth)s"'

sys.path.append(os.getcwd()+"/..")

import dcgan

import theano.tensor as T
import theano
import lasagne

from tasks import check
from scripts import manifold
from lasagne import layers as L

from lasagnekit.misc.plot_weights import dispims_color, tile_raster_images
import matplotlib.pyplot as plt
from skimage.io import imsave

from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.manifold import TSNE
from image_scatter import image_scatter
from tqdm import tqdm

%matplotlib inline
from IPython.display import Image

import matplotlib.pyplot as plt
from lasagnekit.misc.plot_weights import tile_raster_images
import numpy as np
from lasagnekit.misc.plot_weights import dispims_color
from skimage.io import imsave
from helpers import salt_and_pepper

import pickle
from lasagne import layers

def load_gan_model(filename):
    import dcgan
    
    data = pickle.load(open(filename))
    gen = data['generator']
    discr = data['discriminator']
    return gen, discr

def sample(x):
    return (np.random.uniform(size=x.shape) <= x).astype(np.float32)

def floatX(x):
    return x.astype(np.float32)

def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, max(len(inputs) - batchsize + 1, len(inputs)), batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

def sigmoid(x):
    return 1./(1. + np.exp(-x))


## load model

In [ ]:
#5a59f5a7094a3b585818e8e2117585c7
v = check(
    what="notebook", 
    filename="../training/initial_models/model_E.pkl",
    #filename="../jobs/results/68fa4141b9f970f6bde95da53b8e54fe/model.pkl", 
    #filename="../training/brush15/model.pkl",
    dataset="digits",
    force_w=28,
    force_h=28,
    #force_c=1,
    #mode='minibatch'
    #force_model_params=dict(use_wta_spatial=False, use_wta_channel=False)
)
capsule, data, layers, w, h, c = v

In [ ]:
for l in layers:
    print(l)

## Only come here To check vertebrate  convnets

In [ ]:
code_layers = ['wta_channel_conv1', 'wta_channel_conv2', 'wta_channel_conv3']
#code_layers = ['wta_channel']
#code_layers = ['wta_channel_conv1', 'wta_channel_conv2']

X = T.tensor4()
encode = theano.function(
    [X], 
    [L.get_output(layers[cc], X) for cc in code_layers]
)

code_t = [T.tensor4() for _ in code_layers]

decode = theano.function(
    code_t,
    L.get_output(layers['output'], {layers[name]: t for name, t in zip(code_layers, code_t)})
)

In [ ]:
X = data.X[0:100]
X=capsule.preprocess(X)

In [ ]:
Xrec = capsule.reconstruct(X)

In [ ]:
C = encode(X)
Xrec = decode(*C)

In [ ]:
C = list(encode(X))
cc = []
for i in cc:
    C[i][:] = 0
"""
M = (1,)
for i in M:
    F = C[i].shape[1]
    W = C[i].shape[2]
    H = C[i].shape[3]
    C[i][np.arange(C[i].shape[0]), 
         np.random.randint(0, F, size=C[i].shape[0]), 
         np.random.randint(0, W, size=C[i].shape[0]), 
         np.random.randint(0, H, size=C[i].shape[0])] = 100
"""

fig = plt.figure(figsize=(20, 20))
Xrec = decode(*C)
recons = dispims_color(Xrec.transpose((0, 2, 3, 1)) * np.ones((1, 1, 1, 3)), border=1, bordercolor=(0.5,0.5,0.5))
plt.imshow(recons)

In [ ]:
orig = dispims_color(X.transpose((0, 2, 3, 1)) * np.ones((1, 1, 1, 3)))
recons = dispims_color(Xrec.transpose((0, 2, 3, 1)) * np.ones((1, 1, 1, 3)), border=1)
imsave('orig.png', orig)
imsave('recons.png', recons)
fig = plt.figure(figsize=(10, 10))
plt.imshow(recons)

In [ ]:
for i in range(256):
    xx = X[10:11]
    a, b, c = encode(xx)
    inds = c.max(axis=(2, 3)) > 0
    inds = np.arange(c.shape[1])[inds[0]]
    
    #for m in range(i):
    #    plt.imshow(c[0, inds[m]])
    #    plt.show()
    c[0, inds[0:i]] = 0
    C = [a, b, c]
    Xrec = decode(*C)

    #orig = dispims_color(xx.transpose((0, 2, 3, 1)) * np.ones((1, 1, 1, 3)))
    
    recons = dispims_color(Xrec.transpose((0, 2, 3, 1)) * np.ones((1, 1, 1, 3)), border=1)
    #imsave('orig.png', orig)
    #imsave('recons.png', recons)
    fig = plt.figure(figsize=(2, 2))
    plt.imshow(recons, interpolation='none')
    plt.show()

## viz features

In [ ]:
nbpixels = 28*28
for name, layer in layers.items():
    if hasattr(layer, "W"):
        W = layer.W.get_value()
        if len(W.shape)==2 and (W.shape[0] == nbpixels or W.shape[1] == nbpixels):
            nbpixels = c*w*h
            if W.shape[0] == nbpixels:
                W = W.T
            W = W.reshape((W.shape[0], c, w, h))
            W = W.transpose((0, 2, 3, 1))
            W = W * np.ones((1, 1, 1, 3))
        elif len(W.shape) == 4 and W.shape[1] in (1, 3):
            W = W.transpose((0, 2, 3, 1))
            W = W * np.ones((1, 1, 1, 3))
        elif len(W.shape) == 4 and W.shape[0] in (1, 3):
            W = W.transpose((1, 2, 3, 0))
            W = W * np.ones((1, 1, 1, 3))
        else:
            print(W.shape)
            continue
        print(layer.name)
        fig = plt.figure(figsize=(10, 10))
        plt.axis('off')
        img = dispims_color(W, border=1)
        plt.imshow(img, interpolation='none')
        plt.show()

## iterative refinement

In [ ]:
from data import load_data
if c == 3:
    external_data = load_data(dataset='lfw', w=w, h=h)
    external_data = external_data.X
    external_data = external_data.reshape((external_data.shape[0], c, w, h))
    
if c == 1:
    from lasagnekit.datasets.rescaled import Rescaled
    external_data = load_data(dataset='digits', w=w, h=h)
    external_data = Rescaled(external_data, (w, h))
    external_data.load()
    external_data = external_data.X[0:100]
    external_data = external_data.reshape((external_data.shape[0], c, w, h))
    external_data = external_data.astype(np.float32)

genetic = False
if genetic:
    external_data = np.random.uniform(size=external_data.shape).astype(np.float32)

    layer_name = 'wta_spatial'
    x = T.tensor4()
    px_to_code = theano.function(
        [x],
        L.get_output(layers[layer_name], x)
    )
    code_to_px = theano.function(
        [x],
        L.get_output(layers["output"], {layers[layer_name]: x}))

    def crossover(A, nb=100):
        S = np.zeros((nb,) + A.shape[1:])
        for i in range(nb):
            ind1, ind2 = np.random.randint(0, A.shape[0], size=2)
            s1, s2 = A[ind1], A[ind2]
            s = np.zeros_like(s1)
            c = np.random.randint(0, 2, size=A.shape[1])
            s[c == 0] = s1[c == 0]
            s[c == 1] = s2[c == 1]
            S[i] = s
        return S.astype(np.float32)

    def smarter_mutation(A, born_perc=0.1, dead_perc=0.1, nbtimes=1, val=10, nb=100):
        # val = A.max()
        perc = born_perc + dead_perc
        nb_filters = A.shape[1]
        size = int(perc * nb_filters)
        for i in range(nbtimes):
            for a in A:
                indices = np.random.choice(np.arange(a.shape[1]),#was a  bug in the old genetic
                                           size=size, replace=True)
                nb_born = int(born_perc*nb_filters)
                born_indices = indices[0:nb_born]
                dead_indices = indices[nb_born:]
                a[dead_indices] = 0
                for idx in born_indices:
                    a[idx] = 0
                    x, y = np.random.randint(a.shape[1]), np.random.randint(a.shape[2])
                    # val = np.random.choice((0.01, 0.1, 1, 10))
                    #val = np.random.uniform()
                    a[idx, x, y] = val
        return A


    codes = px_to_code(external_data)
    codes = crossover(codes, nb=codes.shape[0])
    codes = smarter_mutation(codes, born_perc=0, dead_perc=0.9)
    external_data = code_to_px(codes)

In [ ]:
seed = 42
seed = np.random.randint(1, 100)
np.random.seed(seed)

N = 25 # nb of examples to generate
nb_iter = 100 # nb iterations for generation

do_sample = False
do_binarize = False
do_noise = False
noise_pr = 0.7
do_gaussian_noise = True
gaussian_noise_std = 0.0001

init_by_external = False


if init_by_external:
    s = floatX(external_data[0:N])
else:
    s = floatX(np.random.uniform(0, 1, size=(N, c, w, h)))

samples = []
samples.append(s.copy()[np.newaxis, :, :, :, :])

nz = np.random.normal(0, gaussian_noise_std, size=s.shape).astype(np.float32)
loss = []
for i in tqdm(range(nb_iter)):
    sprev = s      
    if do_noise:
        s = (np.random.uniform(size=s.shape) <= (1 - noise_pr)) * s
        s = s.astype(np.float32)
    if do_gaussian_noise:
        nz = np.random.normal(0, gaussian_noise_std, size=s.shape).astype(np.float32)
        s += nz
    s = (capsule.reconstruct(s))

    if do_sample:
        s = np.random.binomial(n=1, p=s, size=s.shape).astype('float32')# binarize by sampling
    if do_binarize:
        s = s > 0.5
    samples.append(s.copy()[np.newaxis, :, :, :, :])

    score = np.abs(s - sprev).sum()
    #print(score)
    if score == 0:
        print('end')
        break
    #print(score)
    loss.append(score)
print(score)
samples = np.concatenate(samples, axis=0) # all samples from all timesteps in an array

In [ ]:
# show images generated at last iteration
for s in (samples[0], samples[-1]):
    s = s * np.ones((1, 3, 1, 1))
    s = s.transpose((0, 2, 3, 1))
    fig = plt.figure(figsize=(10, 10))
    img = dispims_color(s, border=2, bordercolor=(0.1, 0.1, 0.1), )
    imsave('out.png', img)
    plt.imshow(img)

In [ ]:
# sample samples and iterations
fig = plt.figure(figsize=(20, 20))
sw, sh = samples.shape[0:2]
samples_ = samples.copy()
samples_ = samples_.reshape((sw*sh, c, w, h))
samples_ = samples_.transpose((0, 2, 3, 1))
samples_ = samples_ * np.ones((1, 1, 1, 3))
img = dispims_color(samples_, shape=(sw, sh))
imsave('out.png', img)
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
import os
from scripts.imgtovideo import imgs_to_video
imgs = []
for s in samples:
    img = dispims_color(s.transpose((0, 2, 3, 1)) * np.ones((1, 1, 1, 3)))
    imgs.append(img)
if os.path.exists('out.mp4'):
    os.remove('out.mp4')
imgs_to_video(imgs, out='out.mp4', verbose=1, framerate=8, rate=8)
embed_video('out.mp4')

In [ ]:
import base64
from IPython.display import HTML
def embed_video(filename):
    video = open(filename, 'r+b').read()
    encoded = base64.b64encode(video)
    return HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                </video>'''.format(encoded.decode('ascii')))


## Video of the last row for brush stroke

In [ ]:
samples[-2].shape

In [ ]:
X = T.tensor4()
encode = theano.function(
    [X], 
    L.get_output(layers['brush'], X)
)

encode_coords = theano.function(
    [X],
    L.get_output(layers['coord'], X)
)

In [ ]:
bias = layers['bias'].b.get_value()

In [ ]:
X = capsule.preprocess(samples[-2][0:16*16])
xx = X.copy()
xx = xx.transpose((0, 2, 3, 1)) * np.ones((1, 1, 1, 3))
source = dispims_color(xx, border=0)

    
y = encode(X)

imgs = []
for t in range(y.shape[1]):
    yy = y[:, t]
    yy = yy[:, :, :, None] * np.ones((1, 1, 1, 3))
    yy = sigmoid(yy+bias) > 0.5
    yy = yy.astype(np.float32)
    img = dispims_color(yy, border=0) 
    plt.imshow(img)
    imgs.append(img)

seq_imgs = imgs

In [ ]:
imgs = [np.zeros_like(imgs[0])] + imgs
img_aggreg = np.zeros((img.shape[0], img.shape[1]*2, 3))
img_aggreg[0:source.shape[0], 0:source.shape[1]] = source
img_aggreg[:, source.shape[1]:source.shape[1]+img.shape[1]] = img
imsave('out.png', img_aggreg)

In [ ]:
import os
if os.path.exists('out.mp4'):
    os.remove('out.mp4')
imgs_to_video(imgs, out='out.mp4', verbose=1, framerate=8, rate=8)

## project into feature space

In [ ]:
hid = []
d = []
for i in range(1):
    d.append(capsule.preprocess(external_data).transpose((0, 2 ,3 , 1))  * np.ones((1, 1, 1, 3)))
    C = encode(capsule.preprocess(external_data))
    if type(C) == list:
        C = [c.reshape(c.shape[0], -1) for c in C]
        C = C[-1]
        #C = np.concatenate(C, axis=1)
        
    hid.append(C)
d = np.concatenate(d, axis=0)
print(hid[0].shape)
hid = np.concatenate(hid, axis=0)
hid = hid.reshape((hid.shape[0], -1))

In [ ]:
mf = PCA(n_components=2)
h2d = mf.fit_transform(hid)

In [ ]:
mf = PCA(n_components=2)
d2d = mf.fit_transform(d.reshape(d.shape[0], -1))

In [ ]:
img = image_scatter(h2d, d, 10)
imsave('out_scatter.png', img)
plt.imshow(img)
plt.show()

In [ ]:
img = image_scatter(d2d, d, 80)
imsave('out_scatter_data.png', img)
plt.imshow(img)
plt.show()

## transferability

In [ ]:
X_transf_orig = external_data[0:500]


X_transf_rec = X_transf_orig.copy().astype(np.float32)
X_transf_rec = X_transf_rec
imgs = []
for i in tqdm(range(100)):
    X_transf_rec = X_transf_rec + (capsule.reconstruct(X_transf_rec) - X_transf_rec)
    imgs.append(X_transf_rec.copy())
    
from scripts import imgtovideo
x = []
for img in imgs:
    img = img * np.ones((1, 3, 1, 1))
    img = img.transpose((0, 2, 3, 1))
    x.append(dispims_color(img))


In [ ]:
imgtovideo.imgs_to_video(x, out='out.mp4', verbose=1)

In [ ]:
X_transf = X_transf_orig.transpose((0, 2, 3, 1))
X_transf = X_transf * np.ones((1, 1, 1, 3))

X_transf_rec = X_transf_rec.transpose((0, 2, 3, 1))
X_transf_rec = X_transf_rec * np.ones((1, 1, 1, 3))

In [ ]:
fig = plt.figure(figsize=(20, 20))
img = dispims_color(X_transf_rec[0:500])
plt.imshow(img, cmap='gray')
plt.show()
fig = plt.figure(figsize=(20, 20))
img = dispims_color(X_transf[0:500])
plt.imshow(img, cmap='gray')
plt.show()

## forced generation

In [ ]:
gen, discr = load_gan_model('/home/mcherti/work/code/lasagne-dcgan/fonts.pkl')

## rnn

In [ ]:
X = T.tensor4()
Xinit = X
nb_steps = 4
for i in range(nb_steps):
    X = L.get_output(layers['output'], X)
score = L.get_output(discr, X)
loss = (score).mean()
grad = theano.grad(loss, Xinit)
get_grad = theano.function([Xinit], grad)
get_loss = theano.function([Xinit], loss)
get_gen = theano.function([Xinit], X)

In [ ]:
import scipy
def eval_loss(x0):
    x0 = x0.reshape((nb, c, w, h))
    return -get_loss(x0.astype(np.float32)).astype('float64')

def eval_grad(x0):
    x0 = x0.reshape((nb, c, w, h))
    return get_grad(x0.astype(np.float32)).flatten().astype('float64')

In [ ]:
nb = 100
x = np.random.uniform(size=(nb, c, w, h))
x = x.astype(np.float32)
for i in range(40):
    #x, _, _ = scipy.optimize.fmin_l_bfgs_b(eval_loss, x.flatten(), fprime=eval_grad, maxfun=1)
    #x = x.reshape((nb, c, w, h)).astype(np.float32)
    x += 0.1 * get_grad(x)
    print(get_loss(x))

In [ ]:
img = get_gen(x) * np.ones((1, 3, 1, 1))
img = img.transpose((0, 2, 3, 1))
fig = plt.figure(figsize=(10, 10))
img = dispims_color(img, border=1, bordercolor=(10, 10, 10))
plt.imshow(img)

In [ ]:
## classif

In [ ]:
from datakit.mnist import MNIST
data = MNIST('train')
data.load()
X_train = data.X.reshape((data.X.shape[0], 1, 28, 28))
y_train = data.y
def transform(X):
    h = []
    for i in range(0, X.shape[0], 128):
        C = encode(X[i:i + 128])
        hh = np.concatenate([cc.max(axis=(2, 3)).reshape((cc.shape[0], -1)) for cc in C], axis=1)      
        h.append(hh)
    h = np.concatenate(h, axis=0)
    return h

X_train = transform(X_train)

data = MNIST('test')
data.load()
X_test = data.X.reshape((data.X.shape[0], 1, 28, 28))
X_test = transform(X_test)
y_test = data.y

In [ ]:
Xrec = capsule.reconstruct(data.)

## fractal low order


In [ ]:
from lasagnekit.datasets.mnist import MNIST
dt = MNIST()
dt.load()

In [ ]:
from IPython import display

W = w * 10
img = np.random.uniform(size=(1, 1, W, W)).astype(np.float32)#>0.7
#img[:, :] = dt.X[0].reshape((W, W))
#img = np.zeros((1, 1, W, W)).astype(np.float32)
#fig = plt.figure(figsize=(5, 5))
for i in range(400):
    x, y = np.random.randint(0, W - w, size=2)
    a = img[:, :, x:x+w, y:y+w].copy()
    for k in range(4):        
        a = capsule.reconstruct(a)
        continue
        vals = a.flatten()
        vals = vals[np.argsort(vals)]
        thresh = vals[-int(0.3 * len(vals)) - 1]

        a= a > thresh


    img[:, :, x:x+w, y:y+w] = a
    
    display.clear_output(wait=True)

    plt.imshow(img[0, 0, w:-w, w:-w], cmap='gray', interpolation='none')
    display.display(plt.gcf())

display.clear_output(wait=True)

In [ ]:
dt_ = capsule.preprocess(data.X[0:100]).copy()

In [ ]:
#dt[:, :, 10:20, 10:20] = 0.
dt = dt_ * (np.random.uniform(size=dt.shape)<=0.5)
rec = capsule.reconstruct(dt)
i = np.random.randint(0, len(dt))
plt.imshow(dt[i, 0], cmap='gray', interpolation='none')
plt.show()
plt.imshow(rec[i, 0], cmap='gray', interpolation='none')
plt.show()

In [ ]:
from skimage.transform import resize
from IPython import display


W = w * 2
img = np.random.uniform(size=(1, 1, W, W)).astype(np.float32)#>0.7
for i in range(100):
    c = np.random.randint(1, w-1)
    x, y = np.random.randint(0, W - c, size=2)

    a = img[:, :, x:x+c, y:y+c]
    a = resize(a[0, 0], (w, h))
    a = a[np.newaxis, np.newaxis, :, :]
    a = a.astype(np.float32)
    for k in range(5):
        a = capsule.reconstruct(a)
    a = resize(a[0, 0], (c, c))
    a = a[np.newaxis, np.newaxis, :, :]
    a = a.astype(np.float32)
    img[:, :, x:x+c, y:y+c] = a#>0.5
    display.clear_output(wait=True)

    plt.imshow(img[0, 0], cmap='gray', interpolation='none')
    display.display(plt.gcf())

display.clear_output(wait=True)

## seq


In [ ]:
from skimage.transform import resize
from skimage.util import pad
T = h * 10
S = np.random.uniform(size=(T, T))
S = S.astype(np.float32)
s=4

fig = plt.figure(figsize=(10, 10))
p = 8
for i in range(100):
    a = pad(S[0:h, i*s:i*s+w], p, 'constant', constant_values=(0, 0))
    a = resize(a, (h, w))
    a = a.astype(np.float32)
    a = a[np.newaxis, np.newaxis, :, :]
    a[:, :, 0, 0:w] = np.random.uniform(size=w)
    for _ in range(1):
        a = capsule.reconstruct(a)
        a = a > 0.5
    m = S[0:h, i*s:(i*s+w)].copy()
    b = a[0, 0, p:-p, p:-p]
    b = resize(b, (h, w))
    #S[0:h, (i*s+w)-s:i*s+w] = b[:, -s:]
    S[0:h, (i*s+w)-s:i*s+w] = b[:, 0:s]
    #S[0:h, i*s:(i*s+w)] = b
    #S[0:h, i*s+w-s:i*s+w] = b[:, i*s+w-s:i*s+w]
    
    display.clear_output(wait=True)
    plt.imshow(m , cmap='gray', interpolation='none')
    plt.imshow(S[0:h], cmap='gray')
    #plt.imshow(b, cmap='gray')
    display.display(plt.gcf())

display.clear_output(wait=True)

In [ ]:
from skimage.transform import resize
def repaint(X):
    if X.shape[0]<=4:
        return X
    m = X.shape[0] / 2
    X[0:m, 0:m] = repaint_(X[0:m, 0:m])
    X[0:m, m:] = repaint_(X[0:m, m:])
    X[m:, 0:m] = repaint_(X[m:, 0:m])
    X[m:, m:] = repaint_(X[m:, m:])
    return X

def repaint_(X):
    shape = X.shape
    X = resize(X, (w, h))
    X = X.astype(np.float32)
    X = rec(X, nb_iter=10)
    X = resize(X, shape)
    X = X.astype(np.float32)
    return repaint(X)
    
def rec(x, nb_iter=4):
    x = x[None, None, :, :]
    for i in range(nb_iter):
        x = capsule.reconstruct(x)
    return x[0, 0]

In [ ]:
T = 28 * 4
S = np.random.uniform(size=(T, T))
S = S.astype(np.float32)
repaint(S)

In [ ]:
fig = plt.figure(figsize=(20, 20))
plt.imshow(S, cmap='gray', interpolation='none')

## VAE

In [ ]:
Z = T.matrix()
X = T.tensor4()
sample = theano.function([Z], L.get_output(layers['output'], {layers['z_sample']: Z}))

pred_z = theano.function([X], L.get_output(layers['z_mu'], X))

z_size = layers['z_sample'].output_shape[1]

In [ ]:
x = data.train.X[0:5000]
y= data.train.y[0:5000]
z = pred_z(capsule.preprocess(x))
fig = plt.figure(figsize=(20, 10))

cmap = plt.get_cmap('jet', 10)
cmap.set_under('gray')

r_inp = np.random.uniform(size=(100, 784)).astype(np.float32)
r = pred_z(capsule.preprocess(r_inp))
plt.scatter(z[:, 0], z[:, 1], c=y, cmap=cmap)
plt.scatter(r[:, 0], r[:, 1], c=[10]*len(r), cmap=cmap, s=100)
plt.colorbar()

In [ ]:
nb = 15*15
if z_size == 2:
    x = np.linspace(-5, 5, np.sqrt(nb))
    y = np.linspace(-5, 5, np.sqrt(nb))
    x, y = np.meshgrid(x, y)
    x = x.flatten()
    y = y.flatten()
    z_samples = np.vstack((x, y)).T
else:
    z_samples = np.random.normal(0, 1, size=(nb, z_size))
    
z_samples = z_samples.astype(np.float32)
x_samples = sample(z_samples)
x_samples = x_samples.transpose((0, 2, 3, 1)) * np.ones((1, 1, 1, 3))
fig = plt.figure(figsize=(10, 10))

img = dispims_color(x_samples)

In [ ]:
imsave('out.png', img)
Image('out.png')

### GAN

In [ ]:
from skimage.transform import resize
import theano

In [ ]:
gen, discr = load_gan_model('../tools/mnist.pkl')

In [ ]:
X = T.tensor4()
discr_fn = theano.function([X], L.get_output(discr, X))

Z = T.matrix()
gen_fn = theano.function([Z], L.get_output(gen, Z))

In [ ]:
z_grad_fn = theano.function([X, Z], theano.grad( ((L.get_output(gen, Z)-X)**2).mean()  , Z))

In [ ]:
x_grad_n = theano.function([X], theano.grad(L.get_output(discr, X).mean(), X))

In [ ]:
def resize_set(x, w, h, **kw):
    x_out = np.empty((x.shape[0], 1, w, h))
    for i in range(len(x)):
        x_out[i, 0] = resize(x[i, 0], (w, h), **kw)
    return x_out.astype(np.float32)

In [ ]:
print(gen.output_shape)

In [ ]:
x = np.random.uniform(size=(9, 1, 28, 28)).astype(np.float32)
Z_of_X = np.random.uniform(-1, 1, size=(x.shape[0], 100)).astype(np.float32)


for i in range(20):
    #x += np.random.normal(0, 0.5, size=x.shape)
    x = capsule.reconstruct(x)
    x = x.astype(np.float32)
    x = resize_set(x, 32, 32)
    if True:
        z_rec = np.random.uniform(-1, 1, size=(x.shape[0], 100)).astype(np.float32)
        for _ in range(10):
            g = z_grad_fn(x, z_rec)
            z_rec -= 0.001*g
        
        gen = gen_fn(z_rec)
        gen =- gen.min(axis=0, keepdims=True)
        gen /= gen.max(axis=0, keepdims=True)
        x = x + (x - gen)
    if False:
        for _ in range(30):
            g = x_grad_n(x)
            x += 0.001*g
    x -= x.min(axis=(1, 2, 3), keepdims=True)
    x /= x.max(axis=(1, 2, 3), keepdims=True) 

    #x -= x.min(axis=0, keepdims=True)
    #x /= x.max(axis=0, keepdims=True) 

    x = resize_set(x, 28, 28, preserve_range=True)
    
    #x = x > 0.3
    x = x.astype(np.float32)

#x = resize_set(x, 32, 32)
print((discr_fn(x)).flatten().mean())

In [ ]:
img =  dispims_color(x.transpose((0, 2, 3, 1)) * np.ones((1, 1, 1, 3)))
imsave('out.png', img)
Image('out.png')